# Spark DataFrames Project Exercise 

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("dataframe_exercise").getOrCreate()

24/07/03 13:13:20 WARN Utils: Your hostname, li-0ab60ccc-31c7-11b2-a85c-cfddf3a09b4d.ibm.com resolves to a loopback address: 127.0.0.1; using 192.168.0.247 instead (on interface enp0s31f6)
24/07/03 13:13:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/03 13:13:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/03 13:13:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/07/03 13:13:22 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/07/03 13:13:22 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [2]:
df = spark.read.csv('walmart_stock.csv',inferSchema=True,header=True)

#### What are the column names?

In [3]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [6]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### Print out the first 5 columns.

In [9]:
df.head(5)

[Row(Date=datetime.date(2012, 1, 3), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996),
 Row(Date=datetime.date(2012, 1, 4), Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475),
 Row(Date=datetime.date(2012, 1, 5), Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539),
 Row(Date=datetime.date(2012, 1, 6), Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922),
 Row(Date=datetime.date(2012, 1, 9), Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)]

#### Use describe() to learn about the DataFrame.

In [16]:
df.describe().show()

24/07/03 13:19:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

In [26]:
df_describe = df.describe()#.filter("summary = mean").show()
df_describe.printSchema()

root
 |-- summary: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [110]:
from pyspark.sql.functions import monotonically_increasing_id, row_number
from pyspark.sql import Window
from pyspark.sql.functions import format_number

df_describe_summary = df_describe.select(df_describe.columns[0]).withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))

df_describe_formated = df_describe.select([format_number(df_describe[c].cast('float') , 2).alias(c) for c in df_describe.columns[1:]])
df_describe_formated_idx = df_describe_formated.select(df_describe_formated.columns).withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))

final_df = df_describe_summary.join(df_describe_formated_idx, df_describe_summary.row_idx == df_describe_formated_idx.row_idx).\
             drop("row_idx")
final_df.show()

+-------+--------+--------+--------+--------+-------------+---------+
|summary|    Open|    High|     Low|   Close|       Volume|Adj Close|
+-------+--------+--------+--------+--------+-------------+---------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|     1,258.00| 1,258.00|
|   mean|   72.36|   72.84|   71.92|   72.39| 8,222,093.50|    67.24|
| stddev|    6.77|    6.77|    6.74|    6.76| 4,519,781.00|     6.72|
|    min|   56.39|   57.06|   56.30|   56.42| 2,094,900.00|    50.36|
|    max|   90.80|   90.97|   89.25|   90.47|80,898,096.00|    84.91|
+-------+--------+--------+--------+--------+-------------+---------+



24/07/03 15:26:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/03 15:26:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/03 15:26:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/03 15:26:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/03 15:26:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/03 15:26:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/07/03 1

#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [111]:
df.show()

+----------+------------------+------------------+------------------+------------------+--------+------------------+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+--------+------------------+
|2012-01-03|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|         59.549999|         58.919998|             59.18| 6679300|51.616215000000004|
|2012-01-10|             59.43|59.709998999999996|             5

In [118]:
new_df = df.withColumn('HV Ratio',(df['High']/df['Volume'])).select(['HV Ratio'])
new_df.show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



#### What day had the Peak High in Price?

In [124]:
df.orderBy("High").show()

+----------+------------------+------------------+------------------+------------------+--------+------------------+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+--------+------------------+
|2015-11-13|56.740002000000004|         57.060001|         56.299999|         56.419998|12514900|53.975581000000005|
|2015-11-02|57.290001000000004|57.610001000000004|             56.77|57.610001000000004|10719200|         55.114026|
|2015-10-28|             57.66|         57.720001|             57.16|57.639998999999996|11834700|         55.142725|
|2015-11-12|57.639998999999996|             57.77|         56.919998|         56.950001| 9551500|54.482620999999995|
|2015-11-16|56.389998999999996|         58.029999|56.360001000000004|         57.869999|13321600|55.362759999999994|
|2015-10-27|58.009997999999996|58.040001000000004|         57.38

#### What is the mean of the Close column?

In [125]:
from pyspark.sql.functions import avg

df.select(avg('Close')).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



#### What is the max and min of the Volume column?

In [130]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

#### How many days was the Close lower than 60 dollars?

In [131]:
df.filter(df["Close"] < 60).describe().show()

+-------+------------------+------------------+------------------+-----------------+--------------------+------------------+
|summary|              Open|              High|               Low|            Close|              Volume|         Adj Close|
+-------+------------------+------------------+------------------+-----------------+--------------------+------------------+
|  count|                81|                81|                81|               81|                  81|                81|
|   mean| 58.95506176543212| 59.37703725925925| 58.54901237037037|58.88827151851851|1.2488990123456791E7| 53.93121002469136|
| stddev|0.7928446895601206|0.7595121127711171|0.7583149556063513|0.736503075015852|   7046459.106735896|2.3480440059467242|
|    min|56.389998999999996|         57.060001|         56.299999|        56.419998|             5340900|         50.363689|
|    max|60.639998999999996|         60.900002|         59.869999|            59.93|            46253500|         57.553275|


#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [139]:
number_day_high = df.filter(df["High"] > 80).count()
total_days = df.count()

percentage = (number_day_high / total_days) * 100
percentage

9.141494435612083

#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [143]:
df.corr('High', 'Volume')

-0.3384326061737161

#### What is the max High per year?

In [148]:
from pyspark.sql.functions import year

#df.withColumn('Year',year(df['Date'])).show()

df.groupBy(year(df['Date'])).agg({"High":'max'}).show()

+----------+---------+
|year(Date)|max(High)|
+----------+---------+
|      2015|90.970001|
|      2013|81.370003|
|      2014|88.089996|
|      2012|77.599998|
|      2016|75.190002|
+----------+---------+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [152]:
from pyspark.sql.functions import month

df.groupBy(month(df['Date'])).agg({"Close":'avg'}).orderBy("month(Date)",).show()

+-----------+-----------------+
|month(Date)|       avg(Close)|
+-----------+-----------------+
|          1|71.44801958415842|
|          2|  71.306804443299|
|          3|71.77794377570092|
|          4|72.97361900952382|
|          5|72.30971688679247|
|          6| 72.4953774245283|
|          7|74.43971943925233|
|          8|73.02981855454546|
|          9|72.18411785294116|
|         10|71.57854545454543|
|         11| 72.1110893069307|
|         12|72.84792478301885|
+-----------+-----------------+



# Great Job!